In [ ]:
### Import Libraries.

import os
import scanpy as sc
import pandas as pd
import numpy as np
import harmonypy as hm
import anndata as ad
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.axes

from pandas.api.types import is_categorical_dtype
from matplotlib.lines import Line2D
from matplotlib.colors import LinearSegmentedColormap

In [ ]:
### Load Data.

os.chdir("/folder/")
adata_all =  ad.read_h5ad("adata.h5ad")

In [ ]:
### Create df for Plot.

df = adata.obs.copy()

df = df[
    (df["Enrichment"] != "Yes") &
    (~df["Status"].isin(["sALS_FTD", "fALS"])) &
    (df["Region"] == "Spinal_Cord")
].copy()

In [ ]:
### Compute Cell Proportions per Sample.

for col in ["Cluster_Column", "Status", "Enrichment"]:
    if is_categorical_dtype(df[col]):
        df[col] = df[col].cat.remove_unused_categories()

total_cells = df.groupby("Sample_ID").size().rename("total_cells")
oligo_cells = (
    df[df["Cluster_Column"] == "Cluster_Value"]
    .groupby("Sample_ID")
    .size()
    .rename("cluster")
)

prop_df = pd.concat([total_cells, oligo_cells], axis = 1).fillna(0)
prop_df["proportion"] = prop_df["cluster"] / prop_df["total_cells"]

sample_status = df[["Sample_ID", "Status"]].drop_duplicates().set_index("Sample_ID")
prop_df = prop_df.join(sample_status)

In [ ]:
### Plot

status_colors = {
    "C9ALS": "#B1746F",
    "Control": "#ADB17D",
    "sALS": "#5B8FA8"
}

plt.figure(figsize = (3, 4))
ax = sns.boxplot(
    data = prop_df,
    x = "Status", y = "proportion",
    order = ["C9ALS", "Control", "sALS"],
    palette = status_colors,
    showfliers = False,
    boxprops = dict(linewidth = 0, alpha = 0.3),
    whiskerprops = dict(linewidth = 1.5, alpha = 0.4),
    capprops = dict(linewidth = 1.5, alpha = 0.4),
    medianprops = dict(linewidth = 1.5, alpha = 0.4)
)

for patch, status in zip(ax.artists, ["C9ALS", "Control", "sALS"]):
    patch.set_facecolor(sns.desaturate(status_colors[status], 0.9))
    patch.set_linewidth(0)

sns.stripplot(
    data = prop_df,
    x = "Status", y = "proportion",
    order = ["C9ALS", "Control", "sALS"],
    palette

for spine in ["top", "right"]:
    ax.spines[spine].set_visible(False)

for spine in ["bottom", "left"]:
    ax.spines[spine].set_linewidth(1.5)
    ax.spines[spine].set_color("#404040")

plt.title("Proportion by Status")
plt.xlabel("")
plt.ylabel("Proportion of Cluster", fontweight = "bold", color = "#404040")
plt.xticks(fontweight = "bold", color = "#404040")
plt.yticks(fontweight = "bold", color = "#404040")
plt.ylim(0, 0.75)
plt.legend([], [], frameon = False)
plt.tight_layout()


plt.savefig("Main_Figure_3_Proportion_Cluster.png", dpi = 800, bbox_inches = 'tight', transparent = True)
plt.show()